###Parsing an NWIS dataset in Pandas
NWIS offers web services that make it convenient to pull down water data for streamflow, wells, and other measurements. 

A full listing of available data and formats can be found from: http://waterdata.usgs.gov/nwis




In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
# urllib is a package for interacting with websites
import urllib

###We will need to process the daily value information at a site to get $Q_{90}$

The site of interest is the Plover River in Wisconsin: 05400513

The URL for pulling daily values for a period of record is:

http://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no=05400513&referred_module=sw&period=&begin_date=2010-01-22&end_date=2015-01-22

###First let's use urllib to download the entire file

This has the advantage of letting us look at the file as a text file and parse some of the goofy NWIS formatting.

Note that we will open the URL and read it down to a text file.

For more information on how to construct these URLs, see:
http://waterservices.usgs.gov/rest/

In [ ]:
dv_URL = "http://waterdata.usgs.gov/nwis/dv?cb_00060=on&format=rdb&site_no=05400513&referred_module=sw&period=&begin_date=2010-01-22&end_date=2015-01-22"
dv_file = urllib.URLopener()
dv_file.retrieve(dv_URL, "DV_Plover_05400513.dat")

### Now let's read this in and check out the first 40 lines

In [ ]:
import os
import numpy as np
NWISfilename = os.path.join("..","Data","12_pandas data",
                                "DV_Plover_05400513.dat")
reconnoiter = open(NWISfilename, 'r').readlines()
for i in np.arange(40):
    print reconnoiter[i].rstrip()


##Test your skillz
How many lines start with '#'?

In [ ]:
numhash = 0 #let's use the as the counter
for line in reconnoiter:
    if line.startswith('#'):
        numhash +=1
    else:
        break
        
print numhash

###To automate reading the file
What is the main challenge here? We can easily tell Pandas that '#' means "comment", but then we have to interpret the column names. 

One way, if we know there are always going to be two header-type lines after the last '#' line, we can just add 2 to numhash, skip that many rows, and read directly from the URL using Pandas `read_csv` function.

In [ ]:
colnames = ['agency','station','date','flow','flag']

nwis_df_url = pd.read_csv(dv_URL,
                          sep = '\s+',
                          skiprows = numhash+2,
                          names = colnames,
                          parse_dates = 2,
                          index_col = 2)

# NOTE We can also read in the text file we downloaded instead
nwis_df_txt = pd.read_csv(NWISfilename,
                          sep = '\s+',
                          skiprows = numhash+2,
                          names = colnames,
                          parse_dates = 2,
                          index_col = 2)


###How does it look?

In [ ]:
nwis_df_url


OK, but we have two non-numeric values. "Dis" and "NaN". 

There's a great method called `dropna` we can use to just remove all rows that have `NaN` values.
http://pandas.pydata.org/pandas-docs/dev/generated/pandas.DataFrame.dropna.html

How can we make all the nonnumeric values into "NaN"?
http://stackoverflow.com/questions/17097236/how-to-replace-values-with-none-in-pandas-data-frame-in-python

NOTE: Both methods have the optional Boolean `inplace` option.

In [ ]:
# so, first replace "Dis" with "NaN"
nwis_df_url.replace('Dis', np.nan, inplace = True)
print '{0} rows in nwis_df_url'.format(len(nwis_df_url))
nwis_df_url.dropna(inplace=True)
print '{0} rows in nwis_df_url'.format(len(nwis_df_url))


###Number conversions
Because we had a mixture of text and numbers, we need to convert flow to float 

In [ ]:
# look at the data types for each column in the DataFrame
nwis_df_url.dtypes

In [ ]:
# convert flow to float (NOTE...when no spaces are in a column name, can use '.')
nwis_df_url.flow=nwis_df_url.flow.astype(float)

#or equivalently
nwis_df_url['flow']=nwis_df_url['flow'].astype(float)


###We can plot with matplotlib

In [ ]:
fig = plt.figure()
plt.plot(nwis_df_url.index,nwis_df_url['flow'])

###Now we can plot the flow over the entire time series with Pandas
Note how much nicer the dates look!

In [ ]:
nwis_df_url['flow'].plot()

##Let's look at things at various time periods
First, we can make a couple new columns, one for year, and one for water year.

How can we group by water year? Not a very easy Google Kung Fu exercise at first, but what about "Fiscal Year"?
Google "Pandas group by fiscal year"
http://stackoverflow.com/questions/26341272/using-groupby-on-pandas-dataframe-to-group-by-financial-year

In [ ]:
# make a year column from the date index
nwis_df_url[u'year'] = nwis_df_url.index.year
#make water year by shifting forward the number of days in Oct., Nov., and Dec.
# NOTE --> shifting by months is less precise
nwis_df_url[u'wateryear'] = nwis_df_url.index.shift(30+31+31,freq='d').year
nwis_df_url[['year','wateryear']].plot()


##Now we can use the groupby method 
Groupby is super powerful and can be used to learn metrics on a year to year basis in this case. We could also, of course, use this to group by other characteristics such as wellname if we had multiple wells, etc.


In [ ]:
print 'Annual Mean'
print nwis_df_url.groupby('year')['flow'].mean()
print '\n\nWater Year Mean'
print nwis_df_url.groupby('wateryear')['flow'].mean()
print '\n\nWater Year 10th percentile'
print nwis_df_url.groupby('wateryear')['flow'].quantile(0.1)
print '\n\nWater Year 90th percentile'
print nwis_df_url.groupby('wateryear')['flow'].quantile(0.9)
print '\n\nWater Year '
print nwis_df_url.groupby('wateryear')['flow'].max()
print '\n\n10th Percentile over entire record'
print nwis_df_url['flow'].quantile(0.1)


###We can plot these as bar charts as well, if we want

In [ ]:
nwis_df_url.groupby('year')['flow'].quantile(0.1).plot(kind='bar', title='year Q90')
plt.figure()
nwis_df_url.groupby('wateryear')['flow'].quantile(0.1).plot(kind='bar', title='wateryear Q90')